In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss
import matplotlib.pyplot as plt
import numpy as np
import torchvision.utils as vutils

In [2]:
# Set the paths for the dataset
data_dir = os.path.join(os.getcwd(), "archive", "chest_xray")
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
val_dir = os.path.join(data_dir, "val")

# Define the dimensions of the input images
input_shape = (1, 512, 512)  # Grayscale images of size 1024x1024

# Define latent dimensions
latent_dim = 100

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Define the VAE model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # Define the encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU()
        )

        # Define the decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, input_shape[0], kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [4]:
# Create data transforms
data_transforms = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((input_shape[1], input_shape[2])),
    transforms.ToTensor()
])

# Load the dataset
train_dataset = ImageFolder(train_dir, transform=data_transforms)
test_dataset = ImageFolder(test_dir, transform=data_transforms)
val_dataset = ImageFolder(val_dir, transform=data_transforms)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# Initialize the model
model = VAE().to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.MSELoss()

# Set random seed for reproducibility
torch.manual_seed(42)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        
        # Forward pass
        recon_batch = model(data)
        
        # Compute the loss
        loss = criterion(recon_batch, data)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if batch_idx % 100 == 99:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, batch_idx+1, len(train_loader), running_loss/100))
            running_loss = 0.0

Epoch [1/10], Step [100/163], Loss: 0.0104
Epoch [2/10], Step [100/163], Loss: 0.0011
Epoch [3/10], Step [100/163], Loss: 0.0007
Epoch [4/10], Step [100/163], Loss: 0.0006
Epoch [5/10], Step [100/163], Loss: 0.0005
Epoch [6/10], Step [100/163], Loss: 0.0004
Epoch [7/10], Step [100/163], Loss: 0.0004
Epoch [8/10], Step [100/163], Loss: 0.0003
Epoch [9/10], Step [100/163], Loss: 0.0003
Epoch [10/10], Step [100/163], Loss: 0.0003


In [10]:
# Generate images using the trained VAE model
with torch.no_grad():
    model.eval()
    for i, (data, _) in enumerate(test_loader):
        data = data.to(device)
        recon_batch = model(data)
        if i == 0:
            save_image(data, os.path.join(data_dir, "original_images.png"))
            save_image(recon_batch, os.path.join(data_dir, "reconstructed_images.png"))

In [11]:
from ignite.metrics import FID, InceptionScore

fid_metric = FID(device=idist.device())
is_metric = InceptionScore(device=idist.device(), output_transform=lambda x: x[0])

NameError: name 'idist' is not defined

In [ ]:
real_batch = next(iter(train_loader))

plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training aImages")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0][:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.show()